In [ ]:
import numpy as np
import warnings
warnings.simplefilter('ignore')
import pandas as pd
%pylab inline
#import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
from sklearn.metrics.regression import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import cross_val_score, train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LogisticRegression,LinearRegression, LassoCV, Lasso 
from sklearn.svm import SVC
from sklearn.metrics import f1_score

classifiers = [LogisticRegression(), 
               RandomForestClassifier(), 
               SVC()]
classifiers_name = ["LogisticRegression",
               "RandomForestClassifier", 
               "SVC"]

In [ ]:
xls_file = pd.ExcelFile('retail_vol_ccy_w_rt.xlsx')
df = xls_file.parse('Total')
df.head()

In [ ]:
df['Date'] = pd.to_datetime(df['Date'], yearfirst=True)

In [ ]:
df.sort_values?

In [ ]:
df.shift

In [ ]:
df['npl_gr'] = df['npl_tot'] - df['npl_tot'].

In [ ]:
plt.plot(df['vol_tot'])
plt.plot(df['expo_tot'])
plt.plot(df['npl_tot'])
plt.show()

In [ ]:
plt.plot(df['rubusd'])
plt.plot(df['rt3m'])
plt.plot(df['wage']/300)
plt.show()

In [ ]:
predictors = list(set(['rubusd','wage','wage_gr','rt3m','dGPD','f_dec', 'unp']))

In [ ]:
df[predictors].hist()

Пока берем в качестве обучающей выборки всю выборку

In [ ]:
X_train = df[predictors]
y_train = df['portf_gr']

X_test = df[predictors].copy()
y_test = df['portf_gr'].copy()

In [ ]:
corr = df[predictors].corr()
corr

In [ ]:
plots = df[predictors].hist()

In [ ]:
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import StratifiedKFold
from sklearn.preprocessing import StandardScaler

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score
forest = RandomForestRegressor(n_estimators=1000, max_depth = 5,
                                random_state=42)
forest.fit(X_train, y_train)
test_pred = forest.predict(X_test)

In [75]:
pd.DataFrame?

In [73]:
forest = RandomForestRegressor(n_estimators=1000, max_depth = 5,
                                random_state=42).fit(X_train, y_train)

features = pd.DataFrame(forest.feature_importances_,
                        index=X_train.columns,columns=['Importance']).sort_values(['Importance'], 
                                                     ascending=False)

features

,Importance
dGPD,0.743125
rubusd,0.100856
rt3m,0.073914
wage,0.053737
wage_gr,0.021560
unp,0.006663
f_dec,0.000144


In [74]:
print(mean_squared_error(forest.predict(X_train), y_train))
print(abs(np.mean(cross_val_score(forest, X_train, y_train, scoring='mean_squared_error', n_jobs=-1))))

1.29998378601e-05
0.000128514781931


Нормируем выборку

In [66]:
X_train.head()

,rt3m,wage,f_dec,rubusd,unp,dGPD,wage_gr
0,16.53,18009,0,33.9032,8.3,-0.035002,0.000000
1,16.04,18007,0,32.9740,8.3,-0.035002,-0.000111
2,12.49,19247,0,30.9843,8.3,-0.035002,0.068862
3,11.81,18872,0,31.0385,8.3,-0.065562,-0.019484
4,11.89,18335,0,31.1533,8.3,-0.065562,-0.028455


In [77]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

In [78]:
X_train_scaled = pd.DataFrame(X_train_scaled,columns=X_train.columns)
X_train_scaled.head()

,rt3m,wage,f_dec,rubusd,unp,dGPD,wage_gr
0,2.085327,-1.642891,-0.290191,-0.536487,2.38862,-1.333206,-0.120777
1,1.952141,-1.643181,-0.290191,-0.598368,2.38862,-1.333206,-0.121738
2,0.987220,-1.463111,-0.290191,-0.730874,2.38862,-1.333206,0.475400
3,0.802390,-1.517568,-0.290191,-0.727265,2.38862,-2.302573,-0.289457
4,0.824135,-1.595550,-0.290191,-0.719620,2.38862,-2.302573,-0.367126


In [67]:
linreg = LinearRegression(n_jobs=-1)
#linreg.fit(X_train_scaled, y_train)
linreg.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=-1, normalize=False)

In [68]:
#print(mean_squared_error(linreg.predict(X_train_scaled), y_train))
print(mean_squared_error(linreg.predict(X_train), y_train))

5.41884523391e-05


In [69]:
linreg_coef = pd.DataFrame(np.c_[linreg.coef_, np.abs(linreg.coef_)], index=df[predictors].columns, 
                           columns=['Coeff.', 'Coeff.abs'])
linreg_coef.sort_values(by='Coeff.abs', ascending=False)

,Coeff.,Coeff.abs
dGPD,2.428705e-01,2.428705e-01
wage_gr,-1.914788e-02,1.914788e-02
f_dec,4.575094e-03,4.575094e-03
unp,-1.266968e-03,1.266968e-03
rt3m,-9.271645e-04,9.271645e-04
rubusd,-2.824829e-04,2.824829e-04
wage,3.073487e-07,3.073487e-07


In [70]:
lasso1 = Lasso(alpha=0.01, random_state=7)
#lasso1.fit(X_train_scaled, y_train)
lasso1.fit(X_train, y_train)
lasso1.coef_

array([ -6.47187679e-04,   9.60784757e-07,  -0.00000000e+00,
        -6.47650991e-04,  -0.00000000e+00,   0.00000000e+00,
        -0.00000000e+00])

In [71]:
lasso1_coef = pd.DataFrame(np.c_[lasso1.coef_, np.abs(lasso1.coef_)], index=df[predictors].columns, 
                           columns=['Coeff.', 'Coeff.abs'])
lasso1_coef.sort_values(by='Coeff.abs', ascending=False)

,Coeff.,Coeff.abs
rubusd,-6.476510e-04,6.476510e-04
rt3m,-6.471877e-04,6.471877e-04
wage,9.607848e-07,9.607848e-07
f_dec,-0.000000e+00,0.000000e+00
unp,-0.000000e+00,0.000000e+00
dGPD,0.000000e+00,0.000000e+00
wage_gr,-0.000000e+00,0.000000e+00


In [ ]:
alphas = np.linspace(0.1, 10, 200)
lasso_cv = LassoCV(alphas=alphas, 
                   cv=5, random_state=7)
lasso_cv.fit(X_train_scaled, y_train)

In [ ]:
lasso_cv_coef = pd.DataFrame(np.c_[lasso_cv.coef_, np.abs(lasso_cv.coef_)], index=df[predictors].columns,  
                           columns=['Coeff.', 'Coeff.abs'])
lasso_cv_coef.sort_values(by='Coeff.abs', ascending=False)

In [72]:
#print(mean_squared_error(lasso_cv.predict(X_train_scaled), y_train))
print(mean_squared_error(lasso_cv.predict(X_train), y_train))

0.000203245084452


In [ ]:
####пример модели
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics
from sklearn.ensemble import ExtraTreesClassifier
from sklearn import cross_validation


def preProcess():
    df= pd.read_csv('C:/Users/X/Desktop/Managerial_and_Decision_Economics_2013_Video_Games_Dataset.csv',encoding ='ISO-8859-1')
    #drop non EA
    df = df[df['EA'] ==1]
    #change categorical variables
    le = LabelEncoder()
    nonnumeric_columns=['Console','Title','Publisher','Genre']
    for feature in nonnumeric_columns:
        df[feature] = le.fit_transform(df[feature])
    #set dataset and target variables
    dataset =df.ix[:, df.columns != 'US Sales (millions)']
    target = df['US Sales (millions)']

    trainx, testx, trainy, testy = cross_validation.train_test_split(
        dataset, target, test_size=0.3, random_state=0)
    #attempt to fix error?
    trainx=np.array(trainx)
    trainy = np.asarray(trainy, dtype="float")
    return trainx,testx,trainy,testy

def classifier():
    model =  ExtraTreesClassifier(n_estimators=250,
                              random_state=0)
    model.fit(trainx,trainy)
    return model.score(testx,testy)


trainx,testx,trainy,testy=preProcess()